In [1]:
import sys
import pandas as pd
import numpy as np
import os
import io
import matplotlib.pyplot as plt
import matplotlib.colors
import json
from urllib.request import urlopen
import dash
import plotly_express as px
import plotly.graph_objects as go
import nbformat
import json
import urllib
import seaborn as sns
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.neighbors import KNeighborsClassifier as KNNc
import sklearn.metrics as Metrics
import re 
import nltk
from shapely.geometry import Point
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.probability import FreqDist
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import pickle
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [2]:
rev= pd.read_csv("/Users/fondairenasantoso/airbnb_data_analysis/data/reviews1.csv")
ny = pd.read_csv("/Users/fondairenasantoso/airbnb_data_analysis/cleaned_airbnb_listings.csv")

In [3]:
merged_rev = ny[['id','property_type','host_id']].merge(rev[['id','listing_id','comments','reviewer_id']],on='id',how='left')

In [4]:
merged_rev = merged_rev.dropna(subset=['comments'], how='any')

In [5]:
merged_rev.head()

,id,property_type,host_id,listing_id,comments,reviewer_id
59,17037.0,Apartment,66035.0,5803.0,A huge thank you to Laurie for 'being there' w...,42873.0
85,23545.0,Townhouse,92411.0,8110.0,Our family of 5 (three nine year old children ...,63099.0
163,45936.0,Bed and breakfast,867225.0,21293.0,Silvia made made my trip perfect. The appartme...,121488.0
306,84010.0,Boutique hotel,417504.0,9782.0,"good place, nice price, everything OK!",185403.0
384,141335.0,House,687361.0,29628.0,We have been very satisfied by the ten days sp...,228033.0


In [6]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [7]:
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
merged_rev['tokenized_column'] = merged_rev.comments.apply(custom_tokenize)

In [8]:
merged_rev.head()

,id,property_type,host_id,listing_id,comments,reviewer_id,tokenized_column
59,17037.0,Apartment,66035.0,5803.0,A huge thank you to Laurie for 'being there' w...,42873.0,"[A, huge, thank, you, to, Laurie, for, 'being,..."
85,23545.0,Townhouse,92411.0,8110.0,Our family of 5 (three nine year old children ...,63099.0,"[Our, family, of, 5, (, three, nine, year, old..."
163,45936.0,Bed and breakfast,867225.0,21293.0,Silvia made made my trip perfect. The appartme...,121488.0,"[Silvia, made, made, my, trip, perfect, ., The..."
306,84010.0,Boutique hotel,417504.0,9782.0,"good place, nice price, everything OK!",185403.0,"[good, place, ,, nice, price, ,, everything, O..."
384,141335.0,House,687361.0,29628.0,We have been very satisfied by the ten days sp...,228033.0,"[We, have, been, very, satisfied, by, the, ten..."


In [9]:
#remove stopwords
merged_rev = merged_rev['tokenized_column'].apply(lambda x: [item for item in x if item not in stops])

In [10]:
merged_rev.shape

(149,)

In [12]:
merged_rev

59       [A, huge, thank, Laurie, 'being, ', much, advi...
85       [Our, family, 5, (, three, nine, year, old, ch...
163      [Silvia, made, made, trip, perfect, ., The, ap...
306      [good, place, ,, nice, price, ,, everything, O...
384      [We, satisfied, ten, days, spent, apartment, C...
                               ...                        
42678    [It, great, stay, apartment, Brooklyn, ., I, w...
43573    [This, apartment, absolutely, beautiful, loved...
48216    [Lee, kind, welcoming, host, ., Her, descripti...
48859    [Ben, 's, place, great, ., Heart, soho, near, ...
49104    [I, would, recommend, room, anyone, I, would, ...
Name: tokenized_column, Length: 149, dtype: object